<a href="https://colab.research.google.com/github/nikbizkit/MMO/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from nltk import tokenize

Загрузим датасет с классификацией записей в сети Твиттер и предполагемой тональностью их содержимого:

In [5]:
df_class = pd.read_csv('comments_twit.csv', sep=",")
df_class.head()

,id,entity,sentiment,comment
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [10]:
# выделим тестовое сообщение, с которым затем будем выполнять задачи предобработки текста
test_val = 9
texts = df_class['comment']
test_text = texts.iloc[test_val]
test_text

'FIX IT JESUS ! Please FIX IT ! What In the world is going on here.  @PlayStation @AskPlayStation @Playstationsup @Treyarch @CallofDuty negative 345 silver wolf error code pic.twitter.com/ziRyhrf59Q'

# Предобработка текста

## Токенизация

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Токенизация по предложениям:

In [12]:
nltk_tk_sents = nltk.tokenize.sent_tokenize(test_text)
print(len(nltk_tk_sents))
nltk_tk_sents

4


['FIX IT JESUS !',
 'Please FIX IT !',
 'What In the world is going on here.',
 '@PlayStation @AskPlayStation @Playstationsup @Treyarch @CallofDuty negative 345 silver wolf error code pic.twitter.com/ziRyhrf59Q']

Токенизация по словам: 

In [13]:
nltk_tk_1 = nltk.WordPunctTokenizer()
nltk_tk_1.tokenize(test_text)

['FIX',
 'IT',
 'JESUS',
 '!',
 'Please',
 'FIX',
 'IT',
 '!',
 'What',
 'In',
 'the',
 'world',
 'is',
 'going',
 'on',
 'here',
 '.',
 '@',
 'PlayStation',
 '@',
 'AskPlayStation',
 '@',
 'Playstationsup',
 '@',
 'Treyarch',
 '@',
 'CallofDuty',
 'negative',
 '345',
 'silver',
 'wolf',
 'error',
 'code',
 'pic',
 '.',
 'twitter',
 '.',
 'com',
 '/',
 'ziRyhrf59Q']

## Частеречная разметка

In [14]:
from spacy.lang.en import English
import spacy
nlp = spacy.load('en_core_web_sm')
spacy_test = nlp(test_text)

Просмотрим какие части речи присутсвуют в тестовом твите:

In [15]:
for token in spacy_test:
    print('{} - {} - {}'.format(token.text, token.pos_, token.dep_))

FIX - PROPN - compound
IT - PROPN - compound
JESUS - PROPN - ROOT
! - PUNCT - punct
Please - INTJ - intj
FIX - VERB - compound
IT - PRON - ROOT
! - PUNCT - punct
What - PRON - pobj
In - ADP - prep
the - DET - det
world - NOUN - pobj
is - AUX - aux
going - VERB - ROOT
on - ADP - prt
here - ADV - advmod
. - PUNCT - punct
  - SPACE - 
@PlayStation - NOUN - dep
@AskPlayStation - PROPN - punct
@Playstationsup - PROPN - compound
@Treyarch - PROPN - ROOT
@CallofDuty - PROPN - punct
negative - ADJ - amod
345 - NUM - nummod
silver - NOUN - compound
wolf - PROPN - compound
error - NOUN - nsubj
code - PROPN - ROOT
pic.twitter.com/ziRyhrf59Q - PUNCT - nummod


## Лемматизация

In [16]:
for token in spacy_test:
      print(token, token.lemma, token.lemma_)

FIX 1140408261210526700 FIX
IT 15566906646452856019 IT
JESUS 8268588004424603548 JESUS
! 17494803046312582752 !
Please 99974891130002844 please
FIX 8299253805695106811 fix
IT 10239237003504588839 it
! 17494803046312582752 !
What 5865838185239622912 what
In 3002984154512732771 in
the 7425985699627899538 the
world 1703489418272052182 world
is 10382539506755952630 be
going 8004577259940138793 go
on 5640369432778651323 on
here 411390626470654571 here
. 12646065887601541794 .
  8532415787641010193  
@PlayStation 4547409949370785885 @playstation
@AskPlayStation 12911826782968708242 @AskPlayStation
@Playstationsup 13814048047829131175 @Playstationsup
@Treyarch 6332745069223603402 @Treyarch
@CallofDuty 3756903042465218067 @CallofDuty
negative 11803922482410560765 negative
345 11235905871042597582 345
silver 4845213853445749913 silver
wolf 3947241998546424405 wolf
error 5141748423479617815 error
code 3084953006211575075 code
pic.twitter.com/ziRyhrf59Q 11188791247425435159 pic.twitter.com/ziRyhr

## Выделение (распознавание) именованных сущностей

In [17]:
for ent in spacy_test.ents:
    print(ent.text, ent.label_)

345 CARDINAL


In [18]:
print(spacy.explain("ORDINAL"))

"first", "second", etc.


In [19]:
print(spacy.explain("PRODUCT"))

Objects, vehicles, foods, etc. (not services)


In [20]:
from spacy import displacy
displacy.render(spacy_test, style='ent', jupyter=True)

## Разбор предложения

In [21]:
displacy.render(spacy_test, style='dep', jupyter=True)

# Решение задачи классификации текста

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

Зададим целевую переменную -- тональность текста:

In [23]:
target = df_class['sentiment'].values
target

array(['Irrelevant', 'Neutral', 'Negative', 'Negative', 'Neutral',
       'Negative', 'Positive', 'Positive', 'Positive', 'Negative',
       'Positive', 'Positive', 'Negative', 'Neutral', 'Negative',
       'Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Negative', 'Neutral', 'Irrelevant', 'Negative', 'Neutral',
       'Neutral', 'Negative', 'Irrelevant', 'Irrelevant', 'Negative',
       'Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Neutral', 'Neutral', 'Irrelevant', 'Positive', 'Neutral',
       'Positive', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral',
       'Negative', 'Negative', 'Negative', 'Neutral', 'Positive',
       'Negative', 'Negative', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Negative', 'Irrelevant', 'Negative',
       'Positive', 'Positive', 'Irrelevant', 'Negative', 'Neutral',
       'Negative', 'Irrelevant', 'Neutral', 'Negative', 'Positive',
       'Negative', 'Negative', 'Positive', 'Positi

## Способ 1. CountVectorizer

In [24]:
countv = CountVectorizer()
countv_features = countv.fit_transform(df_class["comment"])
countv_features

<1000x5440 sparse matrix of type '<class 'numpy.int64'>'
	with 19225 stored elements in Compressed Sparse Row format>

In [25]:
%%time
score_count_svc = cross_val_score(LinearSVC(), countv_features, target, scoring='accuracy', cv=3).mean()

print('Модель векторизации - Countvectorizer, \nМодель классификации - LinearSVC, \nЗначение accuracy = {}'.format(score_count_svc))

Модель векторизации - Countvectorizer, 
Модель классификации - LinearSVC, 
Значение accuracy = 0.4589919260577943
CPU times: user 71 ms, sys: 412 µs, total: 71.5 ms
Wall time: 74.7 ms


Получаем достаточно плохой результат. Возможно это следствие особенности текстов в твиттере с ограничением на количество символов, что приводит к сильным сокращениям и искажениям слов. Вообще лексика неформального общения не совсем совпадает со стандартной.

## Способ 2. word2vec

In [26]:
import gensim
from gensim.models import word2vec

In [27]:
import re
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
# Подготовим корпус
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in df_class['comment'].values:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

In [30]:
corpus[:5]

[['mentioned',
  'facebook',
  'struggling',
  'motivation',
  'go',
  'run',
  'day',
  'translated',
  'tom',
  'great',
  'auntie',
  'hayley',
  'get',
  'bed',
  'told',
  'grandma',
  'thinks',
  'lazy',
  'terrible',
  'person'],
 ['bbc',
  'news',
  'amazon',
  'boss',
  'jeff',
  'bezos',
  'rejects',
  'claims',
  'company',
  'acted',
  'like',
  'drug',
  'dealer',
  'bbc',
  'co',
  'uk',
  'news',
  'av',
  'busine'],
 ['microsoft',
  'pay',
  'word',
  'functions',
  'poorly',
  'samsungus',
  'chromebook'],
 ['csgo',
  'matchmaking',
  'full',
  'closet',
  'hacking',
  'truly',
  'awful',
  'game'],
 ['president',
  'slapping',
  'americans',
  'face',
  'really',
  'commit',
  'unlawful',
  'act',
  'acquittal',
  'discover',
  'google',
  'vanityfair',
  'com',
  'news']]

Обучаем модель word2vec на нашем корпусе

In [31]:
%time model_dz = word2vec.Word2Vec(corpus, workers=4, min_count=10, window=10, sample=1e-3)

CPU times: user 135 ms, sys: 5.14 ms, total: 140 ms
Wall time: 128 ms


In [32]:
# Проверим, что модель обучилась
print(model_dz.wv.most_similar(positive=['find'], topn=5))

[('https', 0.973280668258667), ('time', 0.9732412695884705), ('work', 0.9727075099945068), ('k', 0.9724386930465698), ('com', 0.9718908667564392)]


In [33]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [34]:
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [35]:
df_class.shape


(1000, 4)

In [36]:
df_class.head()

,id,entity,sentiment,comment
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [38]:
dz_df = pd.concat([df_class["comment"], df_class["sentiment"]], axis = 1)

In [39]:
dz_df.head()

,comment,sentiment
0,I mentioned on Facebook that I was struggling ...,Irrelevant
1,BBC News - Amazon boss Jeff Bezos rejects clai...,Neutral
2,@Microsoft Why do I pay for WORD when it funct...,Negative
3,"CSGO matchmaking is so full of closet hacking,...",Negative
4,Now the President is slapping Americans in the...,Neutral


In [50]:
# Обучающая и тестовая выборки
boundary = 200
X_train = corpus[:boundary] 
X_test = corpus[boundary:]
y_train = dz_df.sentiment.values[:boundary]
y_test = dz_df.sentiment.values[boundary:]

In [51]:
%%time
sentiment(EmbeddingVectorizer(model_dz.wv), LogisticRegression(C=5.0))

Метка 	 Accuracy
Irrelevant 	 0.0
Negative 	 0.33962264150943394
Neutral 	 0.8755555555555555
Positive 	 0.0
CPU times: user 94.3 ms, sys: 11.7 ms, total: 106 ms
Wall time: 103 ms


Результаты, полученные с помощью word2vec тоже не очень хоршие, скорее всего здесь нестандартность лексики ещё больше влияет на работу уже предобученной на более-менее формальных корпусах модели. Короткие неформальные сообщения скорее всего требуют немного других подходов.  